# Sentiment Analysis on Movie Reviews using LSTM and Keras
<hr>
<i>Importing libraries</i>

In [1]:
import pandas as pd    # to load dataset
import numpy as np     # for mathematic equation
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.corpus import stopwords   # to get collection of stopwords
from sklearn.model_selection import train_test_split       # for splitting dataset
from tensorflow.keras.preprocessing.text import Tokenizer  # to encode text to int
from tensorflow.keras.preprocessing.sequence import pad_sequences   # to do padding or truncating
from tensorflow.keras.models import Sequential     # the model
from tensorflow.keras.layers import Embedding, LSTM, Dense # layers of the architecture
from tensorflow.keras.callbacks import ModelCheckpoint   # save model
from tensorflow.keras.models import load_model   # load saved model
import re
import matplotlib.pyplot as plt

<hr>
<i>Preview dataset</i>

In [2]:
data = pd.read_csv('IMDB Dataset.csv')

print(data)

                                                  review sentiment
0      One of the other reviewers has mentioned that ...  positive
1      A wonderful little production. <br /><br />The...  positive
2      I thought this was a wonderful way to spend ti...  positive
3      Basically there's a family where a little boy ...  negative
4      Petter Mattei's "Love in the Time of Money" is...  positive
...                                                  ...       ...
49995  I thought this movie did a down right good job...  positive
49996  Bad plot, bad dialogue, bad acting, idiotic di...  negative
49997  I am a Catholic taught in parochial elementary...  negative
49998  I'm going to have to disagree with the previou...  negative
49999  No one expects the Star Trek movies to be high...  negative

[50000 rows x 2 columns]


In [3]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [4]:
english_stops = set(stopwords.words('english'))

<hr>

### Load and Clean Dataset & Encode Sentiments

<hr>

In [5]:
def load_dataset():
    df = pd.read_csv('IMDB Dataset.csv')
    x_data = df['review']       # Reviews/Input
    y_data = df['sentiment']    # Sentiment/Output

    # PRE-PROCESS REVIEW
    x_data = x_data.replace({'<.*?>': ''}, regex = True)          # remove html tag
    x_data = x_data.replace({'[^A-Za-z]': ' '}, regex = True)     # remove non alphabet
    x_data = x_data.apply(lambda review: [w for w in review.split() if w not in english_stops])  # remove stop words
    x_data = x_data.apply(lambda review: [w.lower() for w in review])   # lower case
    
    # ENCODE SENTIMENT -> 0 & 1
    y_data = y_data.replace('positive', 1)
    y_data = y_data.replace('negative', 0)

    return x_data, y_data

x_data, y_data = load_dataset()

print('Reviews')
print(x_data, '\n')
print('Sentiment')
print(y_data)

Reviews
0        [one, reviewers, mentioned, watching, oz, epis...
1        [a, wonderful, little, production, the, filmin...
2        [i, thought, wonderful, way, spend, time, hot,...
3        [basically, family, little, boy, jake, thinks,...
4        [petter, mattei, love, time, money, visually, ...
                               ...                        
49995    [i, thought, movie, right, good, job, it, crea...
49996    [bad, plot, bad, dialogue, bad, acting, idioti...
49997    [i, catholic, taught, parochial, elementary, s...
49998    [i, going, disagree, previous, comment, side, ...
49999    [no, one, expects, star, trek, movies, high, a...
Name: review, Length: 50000, dtype: object 

Sentiment
0        1
1        1
2        1
3        0
4        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: sentiment, Length: 50000, dtype: int64


<hr>

### Spliting Dataset

In [6]:
x_train, x_test, y_train, y_test = train_test_split(x_data, y_data, test_size = 0.2)

print('Train Set')
print(x_train, '\n')
print(x_test, '\n')
print('Test Set')
print(y_train, '\n')
print(y_test)

Train Set
13774    [some, gorehound, friends, recommended, live, ...
44581    [one, underrated, comedies, dan, akroyd, hilar...
48037    [this, movie, lots, potential, beautiful, wome...
37054    [this, movie, heart, felt, piece, cinema, help...
21392    [i, thought, film, could, bit, complex, psycho...
                               ...                        
44643    [i, sure, folks, texas, louisiana, border, mus...
17865    [first, let, agree, lorenzo, lamas, could, nev...
12833    [i, seen, film, many, years, ago, made, lastin...
35697    [i, actually, caught, ad, japan, sinks, japane...
47858    [i, remember, seeing, promos, show, appeared, ...
Name: review, Length: 40000, dtype: object 

18577    [considering, film, reputation, truly, worst, ...
23874    [a, brash, self, centered, army, cadet, arrive...
21878    [wow, i, thought, eskimo, limon, awful, embarr...
37398    [this, movie, go, one, funniest, movies, histo...
40984    [dr, hackenstein, begins, turn, last, century,...
 

<hr>
<i>Function for getting the maximum review length, by calculating the mean of all the reviews length (using <b>numpy.mean</b>)</i>

In [7]:
def get_max_length():
    review_length = []
    for review in x_train:
        review_length.append(len(review))

    return int(np.ceil(np.mean(review_length)))

<hr>

### Tokenize and Pad/Truncate Reviews


In [8]:
# ENCODE REVIEW
token = Tokenizer(lower=False)    # no need lower, because already lowered the data in load_data()
token.fit_on_texts(x_train)
x_train = token.texts_to_sequences(x_train)
x_test = token.texts_to_sequences(x_test)

max_length = get_max_length()

x_train = pad_sequences(x_train, maxlen=max_length, padding='post', truncating='post')
x_test = pad_sequences(x_test, maxlen=max_length, padding='post', truncating='post')

total_words = len(token.word_index) + 1   # add 1 because of 0 padding

print('Encoded X Train\n', x_train, '\n')
print('Encoded X Test\n', x_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[  383 35769   261 ...  6809 10937    73]
 [    5  2061  1211 ...     0     0     0]
 [    8     3   631 ...     0     0     0]
 ...
 [    1    37     4 ... 39881  2864 11636]
 [    1    75   916 ...   352  2884  6708]
 [    1   288   222 ...    44 13242  1070]] 

Encoded X Test
 [[  986     4  2459 ...  1389   731  3329]
 [   40 12087   444 ...  3362   261    41]
 [ 1220     1   102 ...     0     0     0]
 ...
 [    1     6    29 ...     0     0     0]
 [ 6769  1276  9229 ... 30935  3041  3263]
 [  889  6007  5951 ...     2   243 18768]] 

Maximum review length:  130


<hr>

### Build Architecture/Model


In [9]:
# ARCHITECTURE
EMBED_DIM = 32
LSTM_OUT = 64

model = Sequential()
model.add(Embedding(total_words, EMBED_DIM, input_length = max_length))
model.add(LSTM(LSTM_OUT))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 130, 32)           2963040   
_________________________________________________________________
lstm (LSTM)                  (None, 64)                24832     
_________________________________________________________________
dense (Dense)                (None, 1)                 65        
Total params: 2,987,937
Trainable params: 2,987,937
Non-trainable params: 0
_________________________________________________________________
None


<hr>

### Training

In [10]:
checkpoint = ModelCheckpoint(
    'models/LSTM.h5',
    monitor='accuracy',
    save_best_only=True,
    verbose=1
)

In [11]:
history = model.fit(x_train, y_train, batch_size = 128, epochs = 30, callbacks=[checkpoint])

Epoch 1/30
313/313 [==============================] - 56s 170ms/step - loss: 0.4798 - accuracy: 0.7336

Epoch 00001: accuracy improved from -inf to 0.73355, saving model to models\LSTM.h5
Epoch 2/30
313/313 [==============================] - 47s 151ms/step - loss: 0.2177 - accuracy: 0.9210

Epoch 00002: accuracy improved from 0.73355 to 0.92098, saving model to models\LSTM.h5
Epoch 3/30
313/313 [==============================] - 48s 152ms/step - loss: 0.1247 - accuracy: 0.9599

Epoch 00003: accuracy improved from 0.92098 to 0.95995, saving model to models\LSTM.h5
Epoch 4/30
313/313 [==============================] - 46s 148ms/step - loss: 0.0740 - accuracy: 0.9787

Epoch 00004: accuracy improved from 0.95995 to 0.97870, saving model to models\LSTM.h5
Epoch 5/30
313/313 [==============================] - 134s 428ms/step - loss: 0.0539 - accuracy: 0.9854

Epoch 00005: accuracy improved from 0.97870 to 0.98540, saving model to models\LSTM.h5
Epoch 6/30
313/313 [===========================

<hr>

### Testing


In [12]:

y_pred = (model.predict(x_test) > 0.5).astype("int32")

true = 0
for i, y in enumerate(y_test):
    if y == y_pred[i]:
        true += 1

print('Correct Prediction: {}'.format(true))
print('Wrong Prediction: {}'.format(len(y_pred) - true))
print('Accuracy: {}'.format(true/len(y_pred)*100))

Correct Prediction: 8651
Wrong Prediction: 1349
Accuracy: 86.50999999999999


<hr>

### Load Saved Model


In [13]:
loaded_model = load_model('models/LSTM.h5')

In [19]:
review = str(input('Movie Review: '))

Movie Review: this is a really intense movie.


In [20]:
# Pre-process input
regex = re.compile(r'[^a-zA-Z\s]')
review = regex.sub('', review)
print('Cleaned: ', review)

words = review.split(' ')
filtered = [w for w in words if w not in english_stops]
filtered = ' '.join(filtered)
filtered = [filtered.lower()]

print('Filtered: ', filtered)

Cleaned:  this is a really intense movie
Filtered:  ['really intense movie']


In [21]:
tokenize_words = token.texts_to_sequences(filtered)
tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
print(tokenize_words)

[[  13 1443    3    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0    0    0    0    0    0    0    0    0    0    0
     0    0    0    0]]


In [22]:
result = model.predict(tokenize_words)
print(result)

[[0.9954866]]


In [23]:
if result >= 0.7:
    print('positive')
else:
    print('negative')

positive
